This notebook explores the OpenAI GYM CartPole environment

The action space has two actions in a nd array, with shape (1, ), that can take on values in {0, 1}

0 - push cart left

1 - push cart right

The observation has four values:

| Num | Observation           | Min                 | Max               |
|-----|-----------------------|---------------------|-------------------|
| 0   | Cart Position         | -4.8                | 4.8               |
| 1   | Cart Velocity         | -Inf                | Inf               |
| 2   | Pole Angle            | ~ -0.418 rad (-24°) | ~ 0.418 rad (24°) |
| 3   | Pole Angular Velocity | -Inf                | Inf               |

In [28]:
%pip install gym
%pip install gym[classic_control]
%pip install tensorflow
%pip install keras-rl2


[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
def randomActions(env):
    for episode in range(10):
        env.reset()
        env.render()
        terminated = False
        score = 0

        while not terminated:
            action = env.action_space.sample()
            _, reward, terminated, _, _ = env.step(action)
            score += reward
        
        print('Episode: {}, Score: {}'.format(episode, score))

    env.close()

In [2]:
def physicsSolution(env):
    for episode in range(5):
        env.reset()
        env.render()
        terminated, truncated = False, False
        score = 0

        action = 0
        while not terminated and not truncated:
            observation, reward, terminated, truncated, _ = env.step(action)
            if abs(observation[2]) < 0.03:
                action = 0 if observation[3] < 0 else 1
            else:
                action = 0 if observation[2] < 0 else 1
            
            score += reward
        
        print('Episode: {}, Score: {}'.format(episode, score))

    env.close()

In [ ]:
import gym

env = gym.make("CartPole-v1", render_mode="human")
physicsSolution(env)

PyTorch DQN Agent

In [8]:
from collections import deque
import random
from scipy.interpolate import interp1d
import torch
import torch.nn as nn
import torch.nn.functional as F

In [10]:
class ActionObservationSequence:

    # Params:
    # initial_observation - initial observation of the environment (x_1)
    # max_steps - limit at which point we end the sequence
    def __init__(self, initial_observation, max_steps):
        self.sequence = deque(maxlen=max_steps)
        self.sequence.append(initial_observation)
    
    def update(self, action, observation):
        self.sequence.append(action)
        self.sequence.append(observation)

In [9]:
class ReplayMemory:
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = deque(maxlen=capacity)
    
    def update(self, transition):
        # transition[0] = state at time t
        # transition[1] = action at time t
        # transition[2] = reward at time t
        # transition[3] = state at time t + 1
        self.memory.append(transition)
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

In [11]:
class DQN(nn.Module):

    def __init__(self, states, actions):
        super().__init__()
        self.layer1 = nn.Linear(states, 24)
        self.layer2 = nn.Linear(24, 48)
        self.layer3 = nn.Linear(48, 24)
        self.layer4 = nn.Linear(24, actions)
    
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        x = F.relu(self.layer4(x))
        return x
    
    # def train

In [12]:
class ExplorationScheduler():

    def __init__(self, epsilon_initial, epsilon_final, num_episodes):
        self.scheduler = interp1d([1, num_episodes], [epsilon_initial, epsilon_final])
    
    # as the current episode increases, epsilon increases from epsilon initial to epsilon final
    def getEpsilon(self, curr_episode):
        return float(self.scheduler(curr_episode))

In [17]:
import gym

env = gym.make("CartPole-v1", render_mode="human")
states = env.observation_space.shape[0]
actions = env.action_space.n

model = DQN(states, actions)
D = ReplayMemory(64)

memory_capacity = 1000
num_episodes = 100
max_seq_len = 512

epsilon_initial = 0.20
epislon_final = 0.95
ex_scheduler = ExplorationScheduler(epsilon_initial=epsilon_initial, epsilon_final=epislon_final, num_episodes=num_episodes)

for episode in range(1, num_episodes):
    state, _ = env.reset()
    terminated = False
    truncated = False

    while not terminated and not truncated:
        if random.random() > ex_scheduler.getEpsilon(episode): # Explore, choose random action
            action = env.action_space.sample()
        else: # Exploit, choose perceived best action
            with torch.no_grad():
                # Convert state to torch tensor
                stateTensor = torch.Tensor(state)
                
                # Pass state through model to get best action
                modelOutput = model.forward(stateTensor)

                # Get model predicted best action
                action = torch.argmax(modelOutput).item()

        next_state, reward, terminated, truncated, _ = env.step(action)
        
        if terminated or truncated:
            D.update([state, action, reward, None])
        else:
            D.update([state, action, reward, next_state])
        
        mini_batch = D.sample(min(len(D.memory), 8))




KeyboardInterrupt: 